In [1]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

import argparse
import torch

from data.utill import get_data

from models.toeknizers import get_tokenizer
from models.bart import get_bart_model
from train.train_utill import prepare_for_finetuning, set_trainer_for_finetuning

In [2]:
decoder_strat_token = ''
end_of_sentence = ''

In [3]:
datas = get_data(decoder_strat_token, end_of_sentence, 'finetuning') # -> finetuning
if len(datas) == 2:
    encoder_input_train, decoder_output_train = datas
elif len(datas) == 3:
    encoder_input_train, decoder_input_train, decoder_output_train = datas

['/opt/ml/git/data/Validation/시사교육.json', '/opt/ml/git/data/Validation/미용과건강.json', '/opt/ml/git/data/Validation/행사.json', '/opt/ml/git/data/Validation/상거래(쇼핑).json', '/opt/ml/git/data/Validation/주거와생활.json', '/opt/ml/git/data/Validation/개인및관계.json', '/opt/ml/git/data/Validation/여가생활.json', '/opt/ml/git/data/Validation/일과직업.json', '/opt/ml/git/data/Validation/식음료.json']


100%|██████████| 35004/35004 [00:00<00:00, 159945.77it/s]


deleted unnecessary char!!


100%|██████████| 35004/35004 [00:00<00:00, 222424.68it/s]


deleted unnecessary tokens (E.g. #@URL#)!!


In [6]:
tk_name = 'mecab'
load_dir = '/opt/ml/git/dialogue_summarization/results/fine_tuning/mecab/checkpoint-17500/'

In [7]:
print('-'*10, 'Load tokenizer', '-'*10,)
tokenizer = get_tokenizer(tk_name)
print('-'*10, 'Load tokenizer complete', '-'*10,)

---------- Load tokenizer ----------
---------- Load tokenizer complete ----------


In [11]:
train_dataset, valid_dataset = prepare_for_finetuning(tokenizer, encoder_input_train, decoder_input_train, decoder_output_train, batch_size=16)

/opt/conda/envs/dialogue/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [14]:
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

In [15]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [19]:
model = get_bart_model(load_dir, vocab_length= int(len(tokenizer)), tokenizer_name=tk_name)
save_dir = os.path.join(load_dir, 'pytorch_model.bin')
state_dict = torch.load(save_dir) 
model.load_state_dict(state_dict)
model = model.to(device)

/opt/conda/envs/dialogue/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [23]:
list(item)

['input_ids',
 'attention_mask',
 'decoder_input_ids',
 'decoder_attention_mask',
 'labels']

In [37]:
dataloader = DataLoader(train_dataset, batch_size=128)

summary = []
ground = []
with torch.no_grad():
    for item in tqdm(dataloader):
        generated_ids = model.generate(input_ids=item['input_ids'].to(device), 
                        no_repeat_ngram_size=2, 
                        early_stopping=True,
                        repetition_penalty=1.5,
                        max_length=50, 
                        num_beams=5,
                    )  
        for g_ids, gt in zip(generated_ids, item['labels']):
            result_pred = tokenizer.decode(g_ids, skip_special_tokens=True)
            result_Gt = tokenizer.decode(gt, skip_special_tokens=True)
            index = result.find('.')
            if index != -1:
                result = result[:index+1]
            summary.append(result_pred)
            ground.append(result_Gt)

  0%|          | 0/274 [00:00<?, ?it/s]

In [38]:
import pandas as pd
df = pd.DataFrame()

df['pred'] = result_pred
df['gt'] = result_Gt

df.to_csv('./preds/mecab_rep.csv', index=False)

In [33]:
from data.rouge import RougeScorer

In [34]:
rouge = RougeScorer()

In [39]:
print(rouge.compute_rouge(result_pred, result_Gt))

0.000,0.000,0.000


In [40]:
result_pred

'앞뒤 앞뒤 성수동 성수동청정기청정기성역성역토스트토스트 교류 교류 옥상 옥상였으면였으면 발랄 발랄 삼척 삼척 돌려줌 돌려줌 가속 가속 운동가 운동가 수강료 수강료 일으키 일으키 울진 울진 태워 태워 연예대상 연예대상 고마웠 고마웠 계모 계모칠칠 빌려 빌려'